In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('high_function').getOrCreate()

23/09/10 15:43:09 WARN Utils: Your hostname, minseok-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/09/10 15:43:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/10 15:43:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 복잡한 데이터 유형을 위한 내장 함수

https://spark.apache.org/docs/latest/api/sql/index.html

책에서는 array, map을 소개했다. 링크에서 array, map따로 찾아서 보자.

## 고차함수

위의 내장함수 외에도 익명 람다 함수를 인수로 사용하는 고차 함수가 있다.

- transform() <br>

배열과 익명함수를 입력하여 사용한다. 각 요소에 익명 함수를 적용한 다음, 결과를 출력 배열에 할당한다.(UDF 접근 방식과 유사하지만 더 효율적이다.)

In [46]:
from pyspark.sql.types import *

In [47]:
schema = StructType([StructField('celsius', ArrayType(IntegerType()))])

t_list = [[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c = spark.createDataFrame(t_list, schema)
t_c.createOrReplaceTempView('tc')

# 데이터 프레임 출력
t_c.show()

+--------------------+
|             celsius|
+--------------------+
|[35, 36, 32, 30, ...|
|[31, 32, 34, 55, 56]|
+--------------------+



In [48]:
# 온도의 배열에 대해 섭씨를 화씨로 계산
spark.sql("""
SELECT celsius, transform(celsius, t->((t*9) div 5 + 32)) AS fagrenheit
FROM tc""").show()

+--------------------+--------------------+
|             celsius|          fagrenheit|
+--------------------+--------------------+
|[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
|[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+



- filter() <br>

입력한 배열의 요소 중 불린 함수가 참인 요소만으로 구성된 배열을 생성한다.

In [49]:
# 온도의 배열에 대해 섭씨 38도 이상을 필터
spark.sql("""
SELECT celsius, filter(celsius, t-> t > 38) AS high
FROM tc""").show()

+--------------------+--------+
|             celsius|    high|
+--------------------+--------+
|[35, 36, 32, 30, ...|[40, 42]|
|[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+



- exists() <br>

입력한 배열의 요소 중 불린 함수를 만족시키는 것이 존재하면 참을 반환한다.

In [50]:
# 온도의 배열에 섭씨 38도의 온도가 있는가?
spark.sql("""
SELECT celsius, exists(celsius, t-> t = 38) AS threshold
FROM tc""").show()

+--------------------+---------+
|             celsius|threshold|
+--------------------+---------+
|[35, 36, 32, 30, ...|     true|
|[31, 32, 34, 55, 56]|    false|
+--------------------+---------+



- reduce() <br>

`reduce(array<T>, B, function<B, T, B>, function<B, R>`

reduce(expr, start, merge, finish) - Applies a binary operator to an initial state and all elements in the array, and reduces this to a single state. The final state is converted into the final result by applying a finish function.

예제를 보고 이해하는 것이 좋다.

In [51]:
spark.sql("""
SELECT reduce(array(1,2,3), 0, (acc, x) -> acc + x)""").show()

# 0: an initial state
# acc: accumulator
# 0 + 1 = 1
# 1 + 2 = 3
# 3 + 3 = 6

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|reduce(array(1, 2, 3), 0, lambdafunction((namedlambdavariable() + namedlambdavariable()), namedlambdavariable(), namedlambdavariable()), lambdafunction(namedlambdavariable(), namedlambdavariable()))|
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                     6|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [56]:
spark.sql("""
SELECT reduce(array(1,2,3), 0, (acc, x) -> acc + x, acc -> acc * 10)""").show()

# 여기서 acc는 이해를 위한 이름이다.
# 어떠한 단어든 사용가능

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|reduce(array(1, 2, 3), 0, lambdafunction((namedlambdavariable() + namedlambdavariable()), namedlambdavariable(), namedlambdavariable()), lambdafunction((namedlambdavariable() * 10), namedlambdavariable()))|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                           60|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 일반적인 데이터 프레임 및 스파크 SQL 작업

스파크 SQL의 기능 중 일부는 데이터 프레임의 다양한 기능에서 유래된다. 작업 목록은 매우 광범위하며 다음을 포함한다.

- 집계 함수
- 수집 함수
- 날짜/시간 함수
- 수학 함수
- 기타 함수
- 비집계 함수
- 정렬 함수
- 문자열 함수
- UDF 함수
- 윈도우 함수
- 참고 - https://spark.apache.org/docs/latest/api/sql/index.html

데이터 준비
1. 공항(airportsna) 정보 데이터, 미국 비행 지연(departureDelays) 데이터를 가져온다.
2. expr()을 사용하여 delay 및 distance 칼럼을 STRING에서 INT로 변환한다.
3. 작은 테이블 foo를 만든다. 작은 시간 범위 동안 시애틀(SEA)에서 출발하여 샌프란시스코(SFO)에 도착하는 3개의 항공편에 대한 정보만 포함되어있다.

In [57]:
from pyspark.sql.functions import expr

In [59]:
# 파일 경로 설정
tripdelaysFilePath = 'departuredelays.csv'
airportsnaFilePath = 'airport-codes-na.txt'

# 공항 데이터세트를 읽어 오기
airportsna = (spark.read
              .format('csv')
              .options(header = 'true', inferShema='true', sep='\t')
              .load(airportsnaFilePath))

airportsna.createOrReplaceTempView('airports_na')

# 출발 지연 데이터세트를 읽어 오기
departureDelays = (spark.read
                   .format('csv')
                   .options(header='true')
                   .load(tripdelaysFilePath))

departureDelays = (departureDelays
                   .withColumn('delay', expr('CAST(delay as INT) as delay'))
                   .withColumn('distance', expr('CAST(distance as INT) as distance')))

departureDelays.createOrReplaceTempView('departureDelays')

# 임시 작은 테이블 생성
foo = (departureDelays
       .filter(expr("""origin == 'SEA' AND destination == 'SFO' and
       date like '01010%' and delay > 0""")))
foo.createOrReplaceTempView('foo')

In [60]:
spark.sql('SELECT * FROM airports_na LIMIT 10').show()

spark.sql('SELECT * FROM departureDelays LIMIT 10').show()

spark.sql('SELECT * FROM foo').show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01011245|    6|     602|   ABE|        ATL|
|01020600|   -8|     369|   ABE|        DTW|
|01021245|   -2|     602|   ABE|        ATL|
|01020605|   -4|     602|   ABE|        ATL|
|01031245|   -4|     602|   ABE|        ATL|
|01030605|    0|     602|   ABE|        ATL|
|01041243|   10|     602|   ABE|        ATL|
|01040605|   28|     602|   ABE|        ATL|
|01051245|   88|     602|   ABE|        AT

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



- Union

동일한 스키마를 가진 두 개의 서로 다른 데이터 프레임을 함께 결합한다.

In [64]:
# 두 테이블 결합
bar = departureDelays.union(foo)
bar.createOrReplaceTempView('bar')

# 결합된 결과 보기(특정 시간 범위에 대한 SEA와 SFO를 필터)
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO'
AND date LIKE '01010%' AND delay > 0""")).show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



bar 데이터 프레임은 foo와 delays의 결합이다.(pandas의 concat!)

In [65]:
# In SQL
spark.sql("""
SELECT *
FROM bar
WHERE origin = 'SEA'
AND destination = 'SFO'
AND date LIKE '01010%'
AND delay > 0
""").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



- join

기본적으로 스파크 SQL 조인은 inner join이며 옵션은 inner, cross, outer, full, full_outer, left, left_outer, right, right_outer, left_semi 및 left_anti이다.

In [69]:
# 출발 지연 데이터(foo)와 공항 정보의 조인
foo.join(airportsna,
         airportsna.IATA == foo.origin)\
.select('City', 'State', 'date', 'delay', 'distance', 'destination').show()

+-------+-----+--------+-----+--------+-----------+
|   City|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



In [70]:
# SQL 예제
spark.sql("""
SELECT a.city, a.State, f.date, f.delay, f.distance, f.destination
FROM foo f
JOIN airports_na a
ON a.IATA = f.origin
""").show()

+-------+-----+--------+-----+--------+-----------+
|   city|State|    date|delay|distance|destination|
+-------+-----+--------+-----+--------+-----------+
|Seattle|   WA|01010710|   31|     590|        SFO|
|Seattle|   WA|01010955|  104|     590|        SFO|
|Seattle|   WA|01010730|    5|     590|        SFO|
+-------+-----+--------+-----+--------+-----------+



- 윈도우

윈도우 함수는 일반적으로 윈도우(입력 행의 범위) 행의 값을 사용하여 다른 행의 형태로 값 집합을 반환한다. 윈도우 함수를 사용하면 모든 입력 행에 대해 단일값을 반환하면서 행 그룹에 대해 작업할 수 있다. 이 섹션에서는 dense_rank()을 사용한다.

https://www.databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html

다음 쿼리에서는 시애틀SEA, 샌프란시스코SFO 및 뉴욕JFK에서 출발하여 특정 목적지 위치로 이동하는 항공편에서 기록된 TotalDelay(sum(Delay)로 계산되는)에 대한 검토부터 시작한다.<br>


이러한 각 출발 공항에 대해 가장 많은 지연이 발생한 3개의 목적지를 찾으려면 어떻게 해야 하는가?

In [80]:
# SQL 예제
spark.sql("""
SELECT origin, destination, TotalDelays, rank
FROM (
SELECT origin, destination, TotalDelays,
dense_rank() OVER (PARTITION BY origin ORDER BY TotalDelays DESC) AS rank
FROM departureDelaysWindow
) t
WHERE rank <= 3
""").show()

# departureDelaysWindow 테이블을 만드는 과정에서 에러가 발생했음. 그냥 코드만 남겨놓는다.

각 윈도우 그룹은 단일 이그제큐터에서 실행될 수 있어야 하며 실행 중에는 단일 파티션으로 구성된다는 점에 유의해야 한다. 그러므로 쿼리가 제한되지 않는지 확인해야 한다.(예: 윈도우 크기 제한)

- 수정

데이터 프레임을 수정하는 것이다.

열추가 - withColumn() 함수를 사용한다.

In [81]:
from pyspark.sql.functions import expr
foo2 = (foo.withColumn(
    'status', expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END")
                  ))

In [82]:
foo2.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|Delayed|
|01010955|  104|     590|   SEA|        SFO|Delayed|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



열 삭제 - drop() 함수를 사용한다.

In [83]:
foo3 = foo2.drop('delay')
foo3.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|Delayed|
|01010955|     590|   SEA|        SFO|Delayed|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



컬럼명 바꾸기 - withColumnRenamed() 함수를 사용한다.

In [84]:
foo4 = foo3.withColumnRenamed('status', 'flight_status')
foo4.show()

+--------+--------+------+-----------+-------------+
|    date|distance|origin|destination|flight_status|
+--------+--------+------+-----------+-------------+
|01010710|     590|   SEA|        SFO|      Delayed|
|01010955|     590|   SEA|        SFO|      Delayed|
|01010730|     590|   SEA|        SFO|      On-time|
+--------+--------+------+-----------+-------------+



피벗

개념 설명을 위해 몇 가지 데이터를 살펴본다.

In [85]:
spark.sql("""SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
FROM departureDelays
WHERE origin = 'SEA'""").show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
|        ORD|    1|    0|
|        DFW|    1|   23|
|        DFW|    1|   36|
|        ORD|    1|  298|
|        JFK|    1|    4|
|        DFW|    1|    0|
|        MIA|    1|    2|
|        DFW|    1|    0|
|        DFW|    1|    0|
|        ORD|    1|   83|
+-----------+-----+-----+
only showing top 20 rows



피벗을 사용하면 month 칼럼에 이름을 배치할 수 있을 뿐만 아니라 (1과 2 대신 각각 Jan과 Feb를 표시할 수 있음) 목적지 및 월별 지연에 대한 집계 계산(이 경우 평균 및 최대)을 수행할 수 있다.

In [86]:
spark.sql("""SELECT * FROM (
SELECT destination, CAST(SUBSTRING(date, 0, 2) AS int) AS month, delay
FROM departureDelays WHERE origin = 'SEA'
)
PIVOT (
CAST(AVG(delay) AS DECIMAL(4,2)) AS AvgDelay, MAX(delay) AS MaxDelay
For month IN (1 JAN, 2 FEB)
)
ORDER BY destination""").show()

+-----------+------------+------------+------------+------------+
|destination|JAN_AvgDelay|JAN_MaxDelay|FEB_AvgDelay|FEB_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       19.86|         316|       11.42|          69|
|        ANC|        4.44|         149|        7.90|         141|
|        ATL|       11.98|         397|        7.73|         145|
|        AUS|        3.48|          50|       -0.21|          18|
|        BOS|        7.84|         110|       14.58|         152|
|        BUR|       -2.03|          56|       -1.89|          78|
|        CLE|       16.00|          27|        null|        null|
|        CLT|        2.53|          41|       12.96|         228|
|        COS|        5.32|          82|       12.18|         203|
|        CVG|       -0.50|           4|        null|        null|
|        DCA|       -1.15|          50|        0.07|          34|
|        DEN|       13.13|         425|       12.95|         625|
|        D